# Enhancing Conversational Agents with Theory of Mind (ToM)
This notebook implements and evaluates four configurations of conversational agents:
1. **No ToM**: baseline LLM with no Theory of Mind reasoning.
2. **First-Order ToM**: models the user's beliefs.
3. **Second-Order ToM**: models what the user believes the agent knows.
4. **Full ToM**: incorporates first- and second-order beliefs and common ground.

We evaluate the agents on two scenarios:
- **Booking Scenario**: implicit conflict in scheduling.
- **Forgotten Purchase Scenario**: explicit conflict with duplicate orders.


## Setup
Import necessary packages, load environment variables, and define the API client.

In [ ]:
import os
import re
import json
import requests
from dataclasses import dataclass
from typing import Dict, Any
from dotenv import load_dotenv
from typing import List
import pandas as pd

load_dotenv()

class OpenRouterClient:
    def __init__(self, model: str = "gpt-oss-20b"):
        self.api_key = os.getenv('API_KEY')
        self.model = model
        self.url = "https://openrouter.ai/api/v1/chat/completions"

    def chat(self, system_prompt: str, user_prompt: str) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        data = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        }

        response = requests.post(self.url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']


## Agent Classes
Define four agent types:
1. **NoToMAgent**: baseline LLM with no ToM reasoning.
2. **FirstOrderTomAgent**: outputs first-order beliefs.
3. **SecondOrderTomAgent**: outputs second-order beliefs.
4. **FullTomAgent**: outputs first-order, second-order, common ground, and response.


In [ ]:

class NoToMAgent:
    """
    Baseline LLM that does not uses ToM.
    """
    def __init__(self):
        self.client = OpenRouterClient()
    
    def respond(self, scenario: Dict[str, Any]) -> str:
        system = (
            "You are an agent assistant."
            "Assist the user with what he or she tells you to do."
            )
        user = (
            f"Existing events: {json.dumps(scenario['existing_events'])}\n"
            f"New request: {scenario['user_request']}\n\n"
        )
        return self.client.chat(system, user)
    

class FirstOrdertTomAgent:
    """
    ToM agent that produces explicit belief traces:
    - FoB (first-order belief about the world/schedule)
    - Response (final natural-language answer to the user)
    """
    def __init__(self):
        self.client = OpenRouterClient()
    
    def respond(self, scenario: Dict[str, Any]) -> str:
        system = (
            "You are an agent assistant that uses Theory of Mind."
            "Assist the user with what he or she tells you to do."
            "Your output must follow this exact format:\n"
            "  FoB: <first-order belief>\n"
            "  Response: <final message>\n\n"
        )
        user = (
            f"Existing events: {json.dumps(scenario['existing_events'])}\n"
            f"New request: {scenario['user_request']}\n"
        )
        return self.client.chat(system, user)
    
    

class SecondOrderTomAgent:
    """
    ToM agent that produces explicit belief traces:
    - SoB (second-order belief: what the user believes/expects)
    """
    def __init__(self):
        self.client = OpenRouterClient()
    
    def respond(self, scenario: Dict[str, Any]) -> str:
        system = (
            "You are an agent assistant that uses Theory of Mind."
            "Assist the user with what he or she tells you to do."
            "Your output must follow this exact format:\n"
            "  SoB: <second-order belief>\n"
            "  Response: <final message>\n\n"
        )
        user = (
            f"Existing events: {json.dumps(scenario['existing_events'])}\n"
            f"New request: {scenario['user_request']}\n"
        )
        return self.client.chat(system, user)
    
class FullTomAgent:
    """
    ToM agent that produces explicit belief traces:
    - FoB (first-order belief about the world/schedule)
    - SoB (second-order belief: what the user believes/expects)
    - Common Ground
    - Response (final natural-language answer to the user)
    """
    def __init__(self):
        self.client = OpenRouterClient()
    
    def respond(self, scenario: Dict[str, Any]) -> str:
        system = (
            "You are an agent assistant that uses Theory of Mind."
            "Assist the user with what he or she tells you to do."
            "Your output must follow this exact format:\n"
            "  FoB: <first-order belief>\n"
            "  SoB: <second-order belief>\n"
            "  Common ground: <shared belief>\n"
            "  Response: <final message>\n\n"
        )
        user = (
            f"Existing events: {json.dumps(scenario['existing_events'])}\n"
            f"New request: {scenario['user_request']}\n"
        )
        return self.client.chat(system, user)

    
    

## Initialize Agents
Create instances of all four agents.

In [ ]:
no_tom_agent = NoToMAgent()
fo_tom_agent = FirstOrdertTomAgent()
so_tom_agent = SecondOrderTomAgent()
full_tom_agent = FullTomAgent()

## Booking Scenario
Test all agents on a scheduling conflict scenario.

In [ ]:
booking_scenario = {
    "name": "Booking",
    "existing_events": [{"title": "Dentist appointment", "time": "Monday 6:30 PM"}],
    "user_request": "Please write in my calendar a dinner with Ana at 7 o'clock on Monday."
}

num_runs = 30
records = []

for run in range(1, num_runs + 1):
    for name, agent in [
        ("NoToM",       no_tom_agent),
        ("FirstOrder",  fo_tom_agent),
        ("SecondOrder", so_tom_agent),
        ("FullToM",  full_tom_agent),
    ]:
        try:
            resp = agent.respond(booking_scenario)
        except Exception as e:
            resp = f"ERROR: {type(e).__name__}: {e}"
        records.append({"run": run, "agent": name, "response": resp})

results_booking_df = pd.DataFrame.from_records(records)
print(results_booking_df)

In [ ]:
results_booking_df.to_excel("agent_runs_booking.xlsx", index=False)

## Forgotten Purchase Scenario
Test all agents on an explicit duplication conflict scenario.


In [ ]:
purchase_scenario = {
    "name": "Forgotten purchase",
    "existing_events": [
        {
            "title": "Recent grocery order (delivery scheduled for today, not editable)",
            "products": "1 shampoo, 1 conditioner, 1 box of cookies, 1 box of milk, 1 bucket of ice cream"
        }
    ],
    "user_request": "Oh! I need to purchase a box of milk, can you help me ordering it?"
}

num_runs = 30
records = []

for run in range(1, num_runs + 1):
    for name, agent in [
        ("NoToM",       no_tom_agent),
        ("FirstOrder",  fo_tom_agent),
        ("SecondOrder", so_tom_agent),
        ("FullToM",  full_tom_agent),
    ]:
        try:
            resp = agent.respond(purchase_scenario)
        except Exception as e:
            resp = f"ERROR: {type(e).__name__}: {e}"
        records.append({"run": run, "agent": name, "response": resp})

results_df_purchase = pd.DataFrame.from_records(records)
print(results_df_purchase)

In [ ]:
results_df_purchase.to_excel("agent_runs_purchase.xlsx", index=False)